# 한국어 Word2Vec 만들기(네이버 영화 리뷰)

In [1]:
!pip install konlpy > /dev/null

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from konlpy.tag import Okt
from gensim.models.word2vec import Word2Vec
import urllib.request

In [53]:
df = pd.read_csv('https://raw.githubusercontent.com/e9t/nsmc/master/ratings.txt', sep='\t')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200000 entries, 0 to 199999
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   id        200000 non-null  int64 
 1   document  199992 non-null  object
 2   label     200000 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 4.6+ MB


In [54]:
df.head(3)

,id,document,label
0,8112052,어릴때보고 지금다시봐도 재밌어요ㅋㅋ,1
1,8132799,"디자인을 배우는 학생으로, 외국디자이너와 그들이 일군 전통을 통해 발전해가는 문화산...",1
2,4655635,폴리스스토리 시리즈는 1부터 뉴까지 버릴께 하나도 없음.. 최고.,1


### 데이터 전처리

In [55]:
# 데이터 중복 여부 확인
df.shape, df.document.nunique()

((200000, 3), 194543)

In [56]:
# 중복 데이터 배제
df.drop_duplicates(subset=['document'], inplace=True)
df.shape

(194544, 3)

In [57]:
# Null 데이터 제거
df.dropna(how='any', inplace=True)
df.shape

(194543, 3)

### 텍스트 전처리

In [58]:
# 한글과 공백 이외는 제거
df['document'] = df.document.str.replace('[^ㄱ-ㅎㅏ-ㅣ가-힣 ]', '')
df.head(3)

,id,document,label
0,8112052,어릴때보고 지금다시봐도 재밌어요ㅋㅋ,1
1,8132799,디자인을 배우는 학생으로 외국디자이너와 그들이 일군 전통을 통해 발전해가는 문화산업...,1
2,4655635,폴리스스토리 시리즈는 부터 뉴까지 버릴께 하나도 없음 최고,1


In [59]:
df['document'].replace('', np.nan, inplace=True)
df.document.isnull().sum()

491

In [60]:
df.dropna(how='any', inplace=True)
df.shape

(194052, 3)

In [61]:
# 불용어 정의
stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']

In [62]:
okt = Okt()

In [63]:
from tqdm.notebook import tqdm
tokenized_data = []
for sentence in tqdm(df.document):
    morphs = okt.morphs(sentence.strip(), stem=True)
    temp_str = [word for word in morphs if word not in stopwords]
    tokenized_data.append(temp_str)

  0%|          | 0/194052 [00:00<?, ?it/s]

In [ ]:
# 리뷰 길이 분포 확인
print('리뷰의 최대 길이 :',max(len(l) for l in tokenized_data))
print('리뷰의 평균 길이 :',sum(map(len, tokenized_data))/len(tokenized_data))
plt.figure(figsize=(12,6))
plt.hist([len(s) for s in tokenized_data], bins=50, color='green', edgecolor='red')
plt.xlabel('length of samples')
plt.ylabel('number of samples')
plt.show()

### Word2Vect 훈련시키기
- size = 워드 벡터의 특징 값. 즉, 임베딩 된 벡터의 차원.
- window = 컨텍스트 윈도우 크기
- min_count = 단어 최소 빈도 수 제한 (빈도가 적은 단어들은 학습하지 않는다.)
- workers = 학습을 위한 프로세스 수
- sg = 0은 CBOW, 1은 Skip-gram.

In [50]:
from gensim.models import Word2Vec
model = %time Word2Vec(sentences = tokenized_data, size=100, window=5, min_count=5, workers=4, sg=0)

In [51]:
model.wv.vectors.size

169300